# Tacit functions
Dyalog APL allows tacit function declaration, which allows the user to apply combinations of functions to implied arguments. _Tacit_ means that the arguments are not explicit. 

In [2]:
f ← +/ ⍝ A tacit definition of the "plus-reduction" (sum) function
f 1 2 3 4

10

***
## Trains
Trains have been available in Dyalog since [version 14.0](http://help.dyalog.com/14.0/Content/RelNotes14.0/Function%20Trains.htm).
![trains](../img/trains.png)

A 2-train is an *atop*:  

> 
```APL
  (g h) ⍵ ⬄ g (  h ⍵)
⍺ (g h) ⍵ ⬄ g (⍺ h ⍵)
```

A 3-train is a *fork*:

>
```APL
  (f g h) ⍵ ⬄ (  f ⍵) g (  h ⍵)
⍺ (f g h) ⍵ ⬄ (⍺ f ⍵) g (⍺ h ⍵)
```

The *left tine* of a fork (but not an atop!) can be an array:

>
```APL
  (A g h) ⍵ ⬄ A g (  h ⍵)
⍺ (A g h) ⍵ ⬄ A g (⍺ h ⍵)
```

### Trains must be in isolation.

In [2]:
2  +,-  5 ⍝ Not a train
2 (+,-) 5 ⍝ Yes a train

¯3

7 ¯3

### Step-by-step evaluation

In [4]:
(+⌿÷≢) 1 2 3 4

2.5

In [5]:
(+⌿ 1 2 3 4) ÷ (≢ 1 2 3 4)

2.5

In [6]:
10 ÷ 4

2.5

### Getting a clearer view
By default, a train will return a tree view of its parts.

In [15]:
]box on -trains=tree
(+⌿,-,×,÷)

Was ON -trains=parens

┌─┼───┐ 
 ⌿ , ┌─┼───┐ 
┌─┘ - , ┌─┼─┐
+ × , ÷

Use the `-trains=parens` boxing option to see the train with parentheses.

In [13]:
]box on -trains=parens
(+⌿,-,×,÷)

Was ON -trains=tree

(+⌿),(-,(×,÷))

***
## Examples

**Plus and minus**

In [3]:
(+,-)2
1 2 3 (+,-) 4
(2 3⍴0)(+,-) 1

2 ¯2

5 6 7 ¯3 ¯2 ¯1

1 1 1 ¯1 ¯1 ¯1
1 1 1 ¯1 ¯1 ¯1

**Top-heavy fraction from a decimal**

In [4]:
(1∧⊢,÷) 1.125

9 8

**Arithmetic mean**

In [2]:
mean←+⌿÷≢
mean 200 4⍴⍳4

1 2 3 4

**Is ⍵ a palindrome?**

In [24]:
(⌽≡⊢)'racecar'

1

**Split delimited text**

In [25]:
','(≠⊆⊢)'comma,delimited,text'

┌─────┬─────────┬────┐
│comma│delimited│text│
└─────┴─────────┴────┘

**Component of a vector in the direction of another vector**  
Sometimes, a train can make the APL nicely resemble its equivalent definition in traditional mathematical notation.

$$\textbf{a}_\textbf{b} = (\textbf{a}\cdot\hat{\textbf{b}})\hat{\textbf{b}}$$

$$\hat{\textbf{b}} = \frac{\textbf{b}}{|\textbf{b}|} $$

In [6]:
Mag←{0.5*⍨+/⍵*2}     ⍝ dfn:   Magnitude of a vector in Euclidean space
Norm←÷∘Mag⍨          ⍝ tacit: Normalised vector
InDirOf←(⊢×+.×)∘Norm ⍝ Component of vector ⍺ in direction of ⍵

***
## Writing tacit functions
Rome wasn't built in a day, and tacit functions aren't usually created in their final form. Often it is best to write a dfn solution and, if the dfn is short and represents a single mathematical or computational idea, consider whether to refine the code into tacit form afterwards.

As an example, let's write a function to calculate the range of values in a numeric array. The range is the difference between the largest value and the smallest.

In [7]:
array←2 3⍴62 31 59 23 26 16 ⍝ Define array

In [9]:
⌈/array   ⍝ Largest value

62 26

We're getting two values because the [reduction](Operators.ipynb#Reduction) is along the last axis (the rows). There are two rows so we get a value for each row.

The ravel function `,` returns a vector of all of the array's elements.

In [10]:
⌈/,array ⍝ Largest value

62

In [13]:
{(⌈/,⍵)-⌊/,⍵}array ⍝ The largest minus the smallest 

46

That works but let's try to *spot the train*.

We are applying 3 functions, so it's a 3-train - a fork.  
`⍺ (f g h) ⍵ ⬄ (⍺ f ⍵) g (⍺ h ⍵)`
```APL
f←⌈/
g←-
h←⌊/
```

First we should factor out ravel
```APL
{(⌈/⍵)-⌊/⍵},array ⍝ Factor out ravel
```

Then we can see the 3 functions in our fork
```APL
{(⌈/-⌊/)⍵},array  ⍝ Almost there
```

Then just remove the dfn, but we're not quite done
```APL
(⌈/-⌊/),array     ⍝ This isn't a complete function
```
This is simply one function `(⌈/-⌊/)` applied after another `,`.
```APL
(⌈/-⌊/)∘,         ⍝ This is a single, complete function
```

In [14]:
range←(⌈/-⌊/)∘,
range array

46